# Slackbot-related work

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from trulens_eval.tru_db import Record, TruDB, LocalSQLite, Chain, Query
from trulens_eval import tru
from trulens_eval.util import TP
from trulens_eval import tru_feedback
from IPython.display import JSON
from ipywidgets import widgets
import json

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


In [3]:
Record._path

('_record',)

In [23]:
conn, c = tru.lms._connect()
c.execute("select * from records")
rows = c.fetchall()
tru.lms._close(conn)
rows

[('record_hash_209b40db6b757f1b55274e299d66cdba',
  'TruBot_relevance',
  'Who is Piotr?',
  " Piotr Mardziel holds Bachelor's and Master's degrees from the Worcester Polytechnic Institute and a PhD in computer science from University of Maryland, College Park. He has conducted post-doctoral research at Carnegie Mellon University, as well as taught classes in trustworthy machine learning at Stanford University and machine learning privacy and security at Carnegie Mellon University.",
  '{"chain": {"question_generator": {"_call": [{"args": {"inputs": {"question": "Who is Piotr?", "chat_history": "Human: Who is Piotr?\\nAI:  Piotr Mardziel is a research and engineering professional who works on transparency and accountability in machine learning with applications to security, privacy, and fairness. He holds Bachelor\\u2019s and Master\\u2019s degrees from the Worcester Polytechnic Institute and a PhD in computer science from University of Maryland, College Park. He has conducted post-doc

In [25]:
conn, c = tru.lms._connect()
c.execute("select * from feedback_defs")
rows = c.fetchall()
tru.lms._close(conn)
rows

[('feedback_hash_c479553ae6ae61dc58f9b9c1d609ca40',
  '{"selectors": {"text": "response"}, "imp_method_name": "not_toxic", "provider": {"class": "Huggingface"}, "feedback_id": "feedback_hash_c479553ae6ae61dc58f9b9c1d609ca40"}'),
 ('feedback_hash_26c84c76d907d808c36028e024af63c0',
  '{"selectors": {"text1": "prompt", "text2": "response"}, "imp_method_name": "language_match", "provider": {"class": "Huggingface"}, "feedback_id": "feedback_hash_26c84c76d907d808c36028e024af63c0"}'),
 ('feedback_hash_90b443be1347d732ededd19cd3968c72',
  '{"selectors": {"prompt": "input", "response": "output"}, "imp_method_name": "relevance", "provider": {"class": "OpenAI", "model_engine": "gpt-3.5-turbo"}, "feedback_id": "feedback_hash_90b443be1347d732ededd19cd3968c72"}'),
 ('feedback_hash_b9475c84f84aee5e117ba7ec7aab7e25',
  '{"selectors": {"question": "input", "statement": ["_record", "chain", "combine_docs_chain", "_call", "args", "inputs", "input_documents"]}, "imp_method_name": "qs_relevance", "provider

In [6]:
from trulens_eval.provider_apis import Endpoint

In [7]:
from langchain.callbacks import get_openai_callback
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms.base import BaseLLM
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.vectorstores import Pinecone
import pinecone

from trulens_eval import tru
from trulens_eval import tru_chain
from trulens_eval.keys import *
from trulens_eval.keys import PINECONE_API_KEY
from trulens_eval.keys import PINECONE_ENV

# Set up GPT-3 model
model_name = "gpt-3.5-turbo"

chain_id = "TruBot_relevance"

# Pinecone configuration.
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV  # next to api key in console
)

identity = lambda h: h

# Embedding needed for Pinecone vector db.
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')  # 1536 dims
docsearch = Pinecone.from_existing_index(
    index_name="llmdemo", embedding=embedding
)
retriever = docsearch.as_retriever()

# LLM for completing prompts, and other tasks.
llm = OpenAI(temperature=0, max_tokens=128)

# Conversation memory.
memory = ConversationSummaryBufferMemory(
    max_token_limit=650,
    llm=llm,
    memory_key="chat_history",
    output_key='answer'
)

# Conversational chain puts it all together.
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    memory=memory,
    get_chat_history=identity,
    max_tokens_limit=4096
)

"""
# Language mismatch fix:
chain.combine_docs_chain.llm_chain.prompt.template = \
    "Use the following pieces of context to answer the question at the end " \
    "in the same language as the question. If you don't know the answer, " \
    "just say that you don't know, don't try to make up an answer.\n\n" \
    "{context}\n\n" \
    "Question: {question}\n" \
    "Helpful Answer: "
"""

# Contexts fix
chain.combine_docs_chain.llm_chain.prompt.template = \
    "Use only the relevant contexts to answer the question at the end " \
    ". Some pieces of context may not be relevant. If you don't know the answer, " \
    "just say that you don't know, don't try to make up an answer.\n\n" \
    "Contexts: \n{context}\n\n" \
    "Question: {question}\n" \
    "Helpful Answer: "

chain.combine_docs_chain.document_prompt.template="\tContext: {page_content}"

# Trulens instrumentation.

In [8]:
hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

f_toxic = tru_feedback.Feedback(hugs.not_toxic).on_response()
f_lang_match = tru_feedback.Feedback(hugs.language_match).on(text1="prompt", text2="response")
f_relevance = tru_feedback.Feedback(openai.relevance).on(prompt="input", response="output")
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance) \
    .on(question="input", statement=Record.chain.combine_docs_chain._call.args.inputs.input_documents) \
    .on_multiple(multiarg="statement", each_query=Query().page_content)

# feedbacks = tru.run_feedback_functions(chain=tc, record=record, feedback_functions=[f_qs_relevance, f_toxic, f_lang_match, f_relevance])

huggingface: 0request [00:00, ?request/s]

openai: 0request [00:00, ?request/s]

In [17]:
tc = tru_chain.TruChain(
    chain,
    chain_id=chain_id,
    feedbacks=[f_toxic, f_lang_match, f_relevance, f_qs_relevance],
    db=tru.lms
)

In [28]:
res, record = tc.call_with_record("What is TruEra?")

started async call


inserting feedback for future evaluation feedback_hash_c479553ae6ae61dc58f9b9c1d609ca40
inserting feedback for future evaluation feedback_hash_26c84c76d907d808c36028e024af63c0
inserting feedback for future evaluation feedback_hash_90b443be1347d732ededd19cd3968c72
inserting feedback for future evaluation feedback_hash_b9475c84f84aee5e117ba7ec7aab7e25
finished async call


In [ ]:
display(record)

In [ ]:
tc.json

In [ ]:
from trulens_eval.tru_feedback import Feedback

In [ ]:
obj = f_qs_relevance.json
display(obj)
#display(Feedback.of_json(obj).to_json())


In [ ]:
# from tinydb import Query
from trulens_eval.tru_db import Query, Record

In [ ]:
dir(Record)

In [ ]:
feedbacks

In [ ]:
for doc in TruDB.project(query=Record.chain.combine_docs_chain._call.args.inputs.input_documents, obj=record):
    print(doc)
    content = TruDB.project(query=Record.page_content, obj=doc)
    print(content)

In [ ]:
feedbacks

In [ ]:
# e = Endpoint(name="openai", rpm=120)
# print(e.pace.qsize())

In [ ]:
tru.endpoint_openai.tqdm.display()
i = 0
while True:
    # print(e.pace.qsize())
    tru.endpoint_openai.pace_me()
    # print(i)
    i+=1

In [ ]:
# tru_feedback.huggingface_language_match(prompt="Hello there?", response="How are you?")

In [ ]:
# db = LocalTinyDB("slackbot.json")
#tru.init_db("slackbot.sql")
#db = LocalSQLite("slackbot.sql.db")
db = LocalSQLite()

In [ ]:
df, dff = db.get_records_and_feedback(chain_ids=[])

In [ ]:
df

In [ ]:
from pprint import PrettyPrinter
pp = PrettyPrinter(compact=True)

for i, row in df.iterrows():
    
    display(widgets.HTML(f"<b>Question:</b> {row.input}"))
    
    display(widgets.HTML(f"<b>Answer:</b> {row.output}"))
    
    details = json.loads(eval(row.details))

    display(widgets.HTML(str(details['chain']['combine_docs_chain']['llm_chain']['prompt']['template'])))
    
    for doc in details['chain']['combine_docs_chain']['_call']['args']['inputs']['input_documents']:
        display(widgets.HTML(f"""
        <div style="border: 1px solid black; padding: 5px;">
        <b>Context chunk</b>: {doc['page_content']}
        """))

        """<br/>

        <b>source</b>: {doc['metadata']['source']}
        </div>"""

    print()


In [ ]:
df = db.select(
    Record,
    Record.record_id,
    Record.chain_id,
    Record.chain._call.args.inputs.question,
    Record.chain._call.rets.answer)

In [ ]:
df

In [ ]:
for row_id, row in df.iterrows():
    record_id = row.record_id
    chain_id = row.chain_id

    main_question = row['Record.chain._call.args.inputs.question']
    main_answer = row['Record.chain._call.rets.answer']

    print(chain_id, record_id, main_question)
    """
    
    print(question, answer)

    # Run feedback function and get value

    feedback = tru.run_feedback_function(
        main_question, main_answer, [
            tru_feedback.get_not_hate_function(
                evaluation_choice='prompt',
                provider='openai',
                model_engine='moderation'
            ),
            tru_feedback.get_sentimentpositive_function(
                evaluation_choice='response',
                provider='openai',
                model_engine='gpt-3.5-turbo'
            ),
            tru_feedback.get_relevance_function(
                evaluation_choice='both',
                provider='openai',
                model_engine='gpt-3.5-turbo'
            )
        ]
    )
    print(f"will insert overall feedback for chain {chain_id}, record {record_id}")
    db.insert_feedback(record_id=record_id, chain_id=chain_id, feedback=feedback)
    """
    
    # display(JSON(row.Record))
    # print(row.Record['chain'])

    model_name = "gpt-3.5-turbo"

    """
    for page in TruDB.project(query=Record.chain.combine_docs_chain._call.args.inputs.input_documents, obj=row.Record):
        answer = page['page_content']
        feedback = tru.run_feedback_function(
            main_question,
            answer,
	        [
            tru_feedback.get_qs_relevance_function(
                evaluation_choice='prompt',
                provider='openai',
                model_engine=model_name
            )]
        )
        db.insert_feedback(record_id=record_id, chain_id=chain_id, feedback=feedback)

    """

    feedback = tru.run_feedback_function(
        main_question, main_answer, [
            tru_feedback.get_language_match_function(
                provider='huggingface'
            )
        ]
    )
    print(f"will insert language match feedback for chain {chain_id}, record {record_id}")
    db.insert_feedback(record_id=record_id, chain_id=chain_id, feedback=feedback)

    # feedback = tru.run_feedback_function(

    #for leaf in TruDB.leafs(row.Record):
    #    print(leaf)


In [ ]:
row.record_id

In [ ]:
feedback = {'openai_hate_function': 1.849137515819166e-05,
 'openai_sentimentpositive_feedback_function': 1,
 'openai_relevance_function': 10}

In [ ]:
db.insert_feedback(2, feedback)

In [ ]:
db.select(Record, table=db.feedbacks)

In [ ]:
model_name = "gpt-3.5-turbo"
feedback = tru.run_feedback_function(
    "Who is Piotr?",
    "Piotr Mardziel works on transparency and accountability in machine learning with applications to security, privacy, and fairness. He holds Bachelor’s and Master’s degrees from the Worcester Polytechnic Institute and a PhD in computer science from University of Maryland, College Park. He has conducted post-doctoral research at Carnegie Mellon University, as well as taught classes in trustworthy machine learning at Stanford University and machine learning privacy and security at Carnegie Mellon University.",
	 [
        tru_feedback.get_qs_relevance_function(
            evaluation_choice='prompt',
            provider='openai',
            model_engine=model_name
        )])

In [ ]:
feedback